# Introduction

We tackle the problem of OCR post processing. In OCR, we map the image form of the document into the text domain. This is done first using an CNN+LSTM+CTC model, in our case based on tesseract. Since this output maps only image to text, we need something on top to validate and correct language semantics.

The idea is to build a language model, that takes the OCRed text and corrects it based on language knowledge. The langauge model could be:
- Char level: the aim is to capture the word morphology. In which case it's like a spelling correction system.
- Word level: the aim is to capture the sentence semnatics. But such systems suffer from the OOV problem.
- Fusion: to capture semantics and morphology language rules. The output has to be at char level, to avoid the OOV. However, the input can be char, word or both.

The fusion model target is to learn:

    p(char | char_context, word_context)

In this workbook we use seq2seq vanilla Keras implementation, adapted from the lstm_seq2seq example on Eng-Fra translation task. The adaptation involves:

- Adapt to spelling correction, on char level
- Pre-train on a noisy, medical sentences
- Fine tune a residual, to correct the mistakes of tesseract 
- Limit the input and output sequence lengths
- Enusre teacher forcing auto regressive model in the decoder
- Limit the padding per batch
- Learning rate schedule
- Bi-directional LSTM Encoder
- Bi-directional GRU Encoder


# Imports

In [1]:
from __future__ import print_function
import tensorflow as tf
import keras.backend as K
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate, GRU
from keras import optimizers
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
from keras.models import load_model
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from autocorrect import spell
import re
%matplotlib inline

Using TensorFlow backend.


# Utility functions

In [2]:
# Limit gpu allocation. allow_growth, or gpu_fraction
def gpu_alloc():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.Session(config=config))

In [3]:
gpu_alloc()

In [4]:
def calculate_WER_sent(gt, pred):
    '''
    calculate_WER('calculating wer between two sentences', 'calculate wer between two sentences')
    '''
    gt_words = gt.lower().split(' ')
    pred_words = pred.lower().split(' ')
    d = np.zeros(((len(gt_words) + 1), (len(pred_words) + 1)), dtype=np.uint8)
    # d = d.reshape((len(gt_words)+1, len(pred_words)+1))

    # Initializing error matrix
    for i in range(len(gt_words) + 1):
        for j in range(len(pred_words) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(gt_words) + 1):
        for j in range(1, len(pred_words) + 1):
            if gt_words[i - 1] == pred_words[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)
    return d[len(gt_words)][len(pred_words)]

In [5]:
def calculate_WER(gt, pred):
    '''

    :param gt: list of sentences of the ground truth
    :param pred: list of sentences of the predictions
    both lists must have the same length
    :return: accumulated WER
    '''
#    assert len(gt) == len(pred)
    WER = 0
    nb_w = 0
    for i in range(len(gt)):
        #print(gt[i])
        #print(pred[i])
        WER += calculate_WER_sent(gt[i], pred[i])
        nb_w += len(gt[i])

    return WER / nb_w

In [6]:
def load_data_with_gt(file_name, num_samples, max_sent_len, min_sent_len, delimiter='\t', gt_index=1, prediction_index=0):
    '''Load data from txt file, with each line has: <TXT><TAB><GT>. The  target to the decoder muxt have \t as the start trigger and \n as the stop trigger.'''
    cnt = 0  
    input_texts = []
    gt_texts = []
    target_texts = []
    for row in open(file_name, encoding='utf8'):
        if cnt < num_samples :
            #print(row)
            sents = row.split(delimiter)
            input_text = sents[prediction_index]
            
            target_text = '\t' + sents[gt_index] + '\n'
            if len(input_text) > min_sent_len and len(input_text) < max_sent_len and len(target_text) > min_sent_len and len(target_text) < max_sent_len:
                cnt += 1
                
                input_texts.append(input_text)
                target_texts.append(target_text)
                gt_texts.append(sents[gt_index])
    return input_texts, target_texts, gt_texts

In [7]:
def load_data(file_name, num_samples, max_sent_len, min_sent_len):
    '''Load data from txt file, with each line has: <TXT><TAB><GT>. The  target to the decoder muxt have \t as the start trigger and \n as the stop trigger.'''
    cnt = 0  
    input_texts = []   
    
    #for row in open(file_name, encoding='utf8'):
    for row in open(file_name):
        if cnt < num_samples :            
            input_text = row           
            if len(input_text) > min_sent_len and len(input_text) < max_sent_len:
                cnt += 1                
                input_texts.append(input_text)
    return input_texts

In [8]:
def vectorize_data(input_texts, max_encoder_seq_length, num_encoder_tokens, vocab_to_int):
    
    if(len(input_texts) > max_encoder_seq_length):
        input_texts = input_texts[:max_encoder_seq_length]
    
    '''Prepares the input text and targets into the proper seq2seq numpy arrays'''
    encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
    
    for i, input_text in enumerate(input_texts):
        for t, char in enumerate(input_text[:max_encoder_seq_length]):
            # c0..cn
            encoder_input_data[i, t] = vocab_to_int[char]
                
    return encoder_input_data

In [9]:
def decode_sequence(input_seq, encoder_model, decoder_model, num_decoder_tokens, max_decoder_seq_length, vocab_to_int, int_to_vocab):
    
    #print(max_decoder_seq_length)
    # Encode the input as state vectors.
    encoder_outputs, h, c  = encoder_model.predict(input_seq)
    states_value = [h,c]
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = vocab_to_int['\t']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    #print(input_seq)
    attention_density = []
    i = 0
    special_chars = ['\\', '/', '-', '—' , ':', '[', ']', ',', '.', '"', ';', '%', '~', '(', ')', '{', '}', '$', '#']
    #special_chars = []
    while not stop_condition:
        #print(target_seq)
        output_tokens, attention, h, c  = decoder_model.predict(
            [target_seq, encoder_outputs] + states_value)
        #print(attention.shape)
        attention_density.append(attention[0][0])# attention is max_sent_len x 1 since we have num_time_steps = 1 for the output
        # Sample a token
        #print(output_tokens.shape)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        
        #print(sampled_token_index)
        sampled_char = int_to_vocab[sampled_token_index]
        
        orig_char = int_to_vocab[int(input_seq[:,i][0])]
        
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
            #print('End', sampled_char, 'Len ', len(decoded_sentence), 'Max len ', max_decoder_seq_length)
            sampled_char = ''
        
        # Copy digits as it, since the spelling corrector is not good at digit corrections
        if(orig_char.isdigit() or orig_char in special_chars):
            decoded_sentence += orig_char            
        else:
            if(sampled_char.isdigit() or sampled_char in special_chars):
                decoded_sentence += ''
            else:
                decoded_sentence += sampled_char
        
        #decoded_sentence += sampled_char


        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]
        
        i += 1
        if(i > 48):
            i = 0
    attention_density = np.array(attention_density)
    
    # Word level spell correct
    '''
    corrected_decoded_sentence = ''
    for w in decoded_sentence.split(' '):
        corrected_decoded_sentence += spell(w) + ' '
    decoded_sentence = corrected_decoded_sentence
    '''
    return decoded_sentence, attention_density


In [10]:
def word_spell_correct(decoded_sentence):
    corrected_decoded_sentence = ''
    special_chars = ['\\', '/', '-', '—' , ':', '[', ']', ',', '.', '"', ';', '%', '~', '(', ')', '{', '}', '$', '#']
    for w in decoded_sentence.split(' '):
        if((len(re.findall(r'\d+', w))==0) and not (w in special_chars)):
            corrected_decoded_sentence += spell(w) + ' '
        else:
            corrected_decoded_sentence += w + ' '
    return corrected_decoded_sentence

# Load data

# Load model params

In [11]:
data_path = '../../dat/'

In [12]:
max_sent_lengths = [50, 100]

In [13]:
vocab_file = {}
model_file = {}
encoder_model_file = {}
decoder_model_file = {}
model = {}
encoder_model = {}
decoder_model = {}
vocab = {}
vocab_to_int = {}
int_to_vocab = {}
max_sent_len = {}
min_sent_len = {}
num_decoder_tokens = {}
num_encoder_tokens = {}
max_encoder_seq_length = {}
max_decoder_seq_length = {}

In [14]:

for i in max_sent_lengths:
    vocab_file[i] = 'vocab-{}.npz'.format(i)
    model_file[i] = 'best_model-{}.hdf5'.format(i)
    encoder_model_file[i] = 'encoder_model-{}.hdf5'.format(i)
    decoder_model_file[i] = 'decoder_model-{}.hdf5'.format(i)
    
    vocab = np.load(file=vocab_file[i])
    vocab_to_int[i] = vocab['vocab_to_int'].item()
    int_to_vocab[i] = vocab['int_to_vocab'].item()
    max_sent_len[i] = vocab['max_sent_len']
    min_sent_len[i] = vocab['min_sent_len']
    input_characters = sorted(list(vocab_to_int))
    num_decoder_tokens[i] = num_encoder_tokens[i] = len(input_characters) #int(encoder_model.layers[0].input.shape[2])
    max_encoder_seq_length[i] = max_decoder_seq_length[i] = max_sent_len[i] - 1#max([len(txt) for txt in input_texts])
    
    model[i] = load_model(model_file[i])
    encoder_model[i] = load_model(encoder_model_file[i])
    decoder_model[i] = load_model(decoder_model_file[i])

/opt/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [15]:
num_samples = 1000000
#tess_correction_data = os.path.join(data_path, 'test_data.txt')
#input_texts = load_data(tess_correction_data, num_samples, max_sent_len, min_sent_len)

OCR_data = os.path.join(data_path, 'new_trained_data.txt')
#input_texts, target_texts, gt_texts = load_data_with_gt(OCR_data, num_samples, max_sent_len, min_sent_len, delimiter='|',gt_index=0, prediction_index=1)
input_texts, target_texts, gt_texts = load_data_with_gt(OCR_data, num_samples, max_sent_len=10000, min_sent_len=0)

In [16]:
# Sample data
print(len(input_texts))
for i in range(10):
    print(input_texts[i], '\n', target_texts[i])

1951
Me dieal Provider Roles: Treating  
 	Medical Provider Roles: Treating


Provider First Name: Christine  
 	Provider First Name: Christine


Provider Last Name: Nolen, MD  
 	Provider Last Name: Nolen, MD


Address Line 1 : 7 25 American Avenue  
 	Address Line 1 : 725 American Avenue


City. W’aukesha  
 	City: Waukesha


StatefProvinee: ‘WI  
 	State/Province: WI


Postal Code: 5 31 88  
 	Postal Code: 53188


Country". US  
 	Country:  US


Business Telephone: (2 62) 92 8- 1000  
 	Business Telephone: (262) 928- 1000


Date ot‘Pirst Visit: 1 2/01f20 17  
 	Date of First Visit: 12/01/2017




In [17]:
# Spell correct before inference
'''
input_texts_ = []
for sent in input_texts:
    sent_ = ''
    for word in sent.split(' '):
        sent_ += spell(word) + ' '
    input_texts_.append(sent_)
input_texts = input_texts_
input_texts_ = []
# Sample data
print(len(input_texts))
for i in range(10):
    print(input_texts[i], '\n', target_texts[i])
'''

"\ninput_texts_ = []\nfor sent in input_texts:\n    sent_ = ''\n    for word in sent.split(' '):\n        sent_ += spell(word) + ' '\n    input_texts_.append(sent_)\ninput_texts = input_texts_\ninput_texts_ = []\n# Sample data\nprint(len(input_texts))\nfor i in range(10):\n    print(input_texts[i], '\n', target_texts[i])\n"

In [18]:
decoded_sentences = []
corrected_sentences = []

#for seq_index in range(len(input_texts)):
results = open('RESULTS.md', 'w')
results.write('|OCR sentence|GT sentence|Char decoded sentence|Word decoded sentence|Sentence length (chars)|\n')
results.write('---------------|-----------|----------------|----------------|----------------|\n')
     

for i, input_text in enumerate(input_texts):
    #print(input_text)
    # Find the input length range to choose the proper model to use
    len_range = max_sent_lengths[-1] # Take the longest range
    for length in max_sent_lengths:
        if(len(input_text) < length):
            len_range = length
            break
    #print(len_range)
    encoder_input_data = vectorize_data(input_texts=[input_text], max_encoder_seq_length=max_encoder_seq_length[len_range], num_encoder_tokens=num_encoder_tokens[len_range], vocab_to_int=vocab_to_int[len_range])
    
    

    target_text = gt_texts[i]
    
    input_seq = encoder_input_data
    #print(input_seq.shape)
    #print(max_decoder_seq_length[len_range])
    #print(max_decoder_seq_length)
    decoded_sentence,_  = decode_sequence(input_seq, encoder_model[len_range], decoder_model[len_range], num_decoder_tokens[len_range],  max_decoder_seq_length[len_range], vocab_to_int[len_range], int_to_vocab[len_range])
    corrected_sentence = word_spell_correct(decoded_sentence)
    print('-Lenght = ', len_range)
    print('Input sentence:', input_text)
    print('GT sentence:', target_text.strip())
    print('Char Decoded sentence:', decoded_sentence)   
    print('Word Decoded sentence:', corrected_sentence) 
    results.write(' | ' + input_text + ' | ' + target_text.strip() + ' | ' + decoded_sentence + ' | ' + corrected_sentence + ' | ' + str(len_range) + ' | \n')
    decoded_sentences.append(decoded_sentence)
    corrected_sentences.append(corrected_sentence)
results.close()    

    

-Lenght =  50
Input sentence: Me dieal Provider Roles: Treating 
GT sentence: Medical Provider Roles: Treating
Char Decoded sentence: Medical Provider Roles:Treating 
Word Decoded sentence: Medical Provider Roles:Treating a 
-Lenght =  50
Input sentence: Provider First Name: Christine 
GT sentence: Provider First Name: Christine
Char Decoded sentence: Provider First Name: Christine 
Word Decoded sentence: Provider First Name Christine a 
-Lenght =  50
Input sentence: Provider Last Name: Nolen, MD 
GT sentence: Provider Last Name: Nolen, MD
Char Decoded sentence: Provider Last Name: Nolen, MD 
Word Decoded sentence: Provider Last Name Dolens MD a 
-Lenght =  50
Input sentence: Address Line 1 : 7 25 American Avenue 
GT sentence: Address Line 1 : 725 American Avenue
Char Decoded sentence: Address Line 1 : 725 Americal Avenuer
Word Decoded sentence: Address Line 1 : 725 America Avenue 
-Lenght =  50
Input sentence: City. W’aukesha 
GT sentence: City: Waukesha
Char Decoded sentence: City.St

-Lenght =  100
Input sentence: TT INSURANCIEINFORMATICN PRIMARY: UMR SECONDARY: '
GT sentence: INSURANCE INFORMATION PRIMARY: UMR SECONDARY:
Char Decoded sentence: INSURANCEN ORN ORMATION PRIMARY:UMMARY UMCOND:RY
Word Decoded sentence: INSURANCE ON FORMATION PRIMARY:UMMARY UMCOND:RY 
-Lenght =  50
Input sentence: STATEMENT DATE 1121/2018
GT sentence: STATEMENT DATE 1/21/2018
Char Decoded sentence: STATEMENT DATE 1121/2
Word Decoded sentence: STATEMENT DATE 1121/2 
-Lenght =  50
Input sentence: PATIENT NAME 
GT sentence: PATIENT NAME
Char Decoded sentence: PATIENT NAME
Word Decoded sentence: PATIENT NAME 
-Lenght =  100
Input sentence: pAvMENTsm'E—I: STATEMENT DATE WILL NOT APPEAR ON THIS STATEMENT 
GT sentence: PAYMENTS RECEIVED AFTER STATE DATE WILL NOT APPEAR ON THIS STATEMENT
Char Decoded sentence: PAYMENTE IN— :TATEMENT DATE WILL NOT APPEAR ON THIS STATEMEN— :ENT 
Word Decoded sentence: PAYMENT IN STATEMENT DATE WILL NOT APPEAR ON THIS STATEMENT WENT a 
-Lenght =  50
Input sentence

-Lenght =  100
Input sentence: To pay your bill on line with a credit card, log on to www.ebixinc.comlpayonline.html. 
GT sentence: To pay your bill on line with a credit card, log on to www.ebixinc.com/payonline.html.
Char Decoded sentence: To pay your bill on line with a credity car, log on to wwwentixincehinccompatoly 
Word Decoded sentence: To pay your bill on line with a credit care log on to wwwentixincehinccompatoly a 
-Lenght =  50
Input sentence: ACCOUNT# EMA297232
GT sentence: ACCOUNT# EMA297232
Char Decoded sentence: ACCOUNT# EMA297232
Word Decoded sentence: ACCOUNT EMA297232 
-Lenght =  100
Input sentence: PLACE OF SERVICE 11 Office 21 Inpatient 22 Outpatient Hospital 23 Emergency Room-Hospital
GT sentence: PLACE OF SERVICE 11 Office 21 Inpatient 22 Outpatient Hospital 23 Emergency Room-Hospital
Char Decoded sentence: PLACE OF ORVERVIC11 Offic21 Oulpatie22 Hospital  Emergency R11m Hospit21  Emergen22
Word Decoded sentence: PLACE OF ORVERVIC11 Offic21 Oulpatie22 Hospital a E

-Lenght =  100
Input sentence: To disclose information, whether from before, during or after the date of this authorization, about my health, including HIV, AIDS or other disorders of the Immune system, use of drugs or alcohol, mental or phy5ica| histor , condition, advice or treatment (except this authorization does not authorize release of psychotherapy notesi, prescription drug history, earnings, financial or credit history, professional licenses, employment history, insurance claims and benefits, and all other claims and benefits, including Social Security claims and benefits (“My Information"); 
GT sentence: To disclose information, whether from before, during or after the date of this authorization, about my health, including HIV, AIDS or other disorders of the immune system, use of drugs or alcohol, mental or physical history, condition, advice or treatment (except this authorization does not authorize release of psychotherapy notes, prescription drug history, earnings, financia

-Lenght =  50
Input sentence: Dependent Information 
GT sentence: Dependent Information
Char Decoded sentence: Dependent Information
Word Decoded sentence: Dependent Information 
-Lenght =  50
Input sentence: First Name: 
GT sentence: First Name:
Char Decoded sentence: First Name: 
Word Decoded sentence: First Name a 
-Lenght =  50
Input sentence: Middle Nameﬂnitial: 
GT sentence: Middle Name/Initial:
Char Decoded sentence: Middle NameInitia: 
Word Decoded sentence: Middle NameInitia: a 
-Lenght =  50
Input sentence: Last Name: 
GT sentence: Last Name:
Char Decoded sentence: Last Name: 
Word Decoded sentence: Last Name a 
-Lenght =  50
Input sentence: Social Security Number: 
GT sentence: Social Security Number:
Char Decoded sentence: Social Security Number: 
Word Decoded sentence: Social Security Number a 
-Lenght =  50
Input sentence: Birth Date: 
GT sentence: Birth Date:
Char Decoded sentence: Birth Date: 
Word Decoded sentence: Birth Date a 
-Lenght =  50
Input sentence: Gender: 
G

...

-Lenght =  50
Input sentence: Middle Name/initial: 
GT sentence: Middle Name/Initial:
Char Decoded sentence: Middle Name/Initial: 
Word Decoded sentence: Middle Name/Initial: a 
-Lenght =  50
Input sentence: Last Name: 
GT sentence: Last Name:
Char Decoded sentence: Last Name: 
Word Decoded sentence: Last Name a 
-Lenght =  50
Input sentence: Social Security Number: 
GT sentence: Social Security Number:
Char Decoded sentence: Social Security Number: 
Word Decoded sentence: Social Security Number a 
-Lenght =  50
Input sentence: Birth Date: 
GT sentence: Birth Date:
Char Decoded sentence: Birth Date: 
Word Decoded sentence: Birth Date a 
-Lenght =  50
Input sentence: Gender: 
GT sentence: Gender:
Char Decoded sentence: Gender: 
Word Decoded sentence: Gender a 
-Lenght =  50
Input sentence: Claim Event Information 
GT sentence: Claim Event Information
Char Decoded sentence: Claim Event Information
Word Decoded sentence: Claim Event Information 
-Lenght =  100
Input sentence: Accident Des

-Lenght =  50
Input sentence: Country". US 
GT sentence: Country: US
Char Decoded sentence: Country".US 
Word Decoded sentence: Country".US a 
-Lenght =  50
Input sentence: Business Telephone: (419) 474- 1210 
GT sentence: Business Telephone: (419) 474-1210
Char Decoded sentence: Business Telephone: (419)474-1210
Word Decoded sentence: Business Telephone (419)474-1210 
-Lenght =  50
Input sentence: Business Fax (419) 474-3076 
GT sentence: Business Fax: (419) 474-3076
Char Decoded sentence: Business Fax (419)474-3076 
Word Decoded sentence: Business Fax (419)474-3076 a 
-Lenght =  50
Input sentence: Date ofFirst Visit: 03/09/2018 
GT sentence: Date of First Visit: 03/09/2018
Char Decoded sentence: Date of First Visi:03/09/2018
Word Decoded sentence: Date of First Visi:03/09/2018 
-Lenght =  50
Input sentence: Date ofNen Visit: 03/16/2018
GT sentence: Date of Next Visit: 03/16/2018
Char Decoded sentence: Date of Next Vis:t03/16/2018
Word Decoded sentence: Date of Next Vis:t03/16/2018 
-

-Lenght =  50
Input sentence: City. 
GT sentence: City:
Char Decoded sentence: City. 
Word Decoded sentence: City a 
-Lenght =  50
Input sentence: State/PrOVirice: 
GT sentence: State/Province:
Char Decoded sentence: State/Province:
Word Decoded sentence: State/Province: 
-Lenght =  50
Input sentence: Postal Code 
GT sentence: Postal Code:
Char Decoded sentence: Postal Code 
Word Decoded sentence: Postal Code a 
-Lenght =  50
Input sentence: Country. 
GT sentence: Country:
Char Decoded sentence: Country. 
Word Decoded sentence: Country a 
-Lenght =  50
Input sentence: Best Phone Number to be Reached During the Day: 
GT sentence: Best Phone Number to be Reached During the Day:
Char Decoded sentence: Best Phone Number to be Reached During the Day:
Word Decoded sentence: Best Phone Number to be Reached During the Day 
-Lenght =  50
Input sentence: Email Address: 
GT sentence: Email Address:
Char Decoded sentence: Email Address: 
Word Decoded sentence: Email Address a 
-Lenght =  50
Input 

-Lenght =  100
Input sentence: Information authorized for use or disclosure may include information which may indicate the presence of a communicable or non-communicable disease. 
GT sentence: Information authorized for use or disclosure may include information which may indicate the presence of a communicable or non-communicable disease.
Char Decoded sentence: Information authorized for disclose may incoude for dislon to hishoral which may indicate the presen
Word Decoded sentence: Information authorized for disclose may include for diplon to hishoral which may indicate the present 
-Lenght =  100
Input sentence: If I do not sign this authorization or if I alter or revoke it, except as specified above, Unum may not be able to evaluate or administer my claim(s), which may lead to my claim(s) being denied. I may revoke this authorization at any time by sending written notice to the address above. I understand that revocation will not apply to any information that Unum requests or disclo

-Lenght =  50
Input sentence: Employee Off—Job Acc June 18, 2012 
GT sentence: Employee Off-Job Acc June 18, 2012
Char Decoded sentence: Employee Off—Job Acc June 18, 2012
Word Decoded sentence: Employee Off—Job Acc June 18, 2012 
-Lenght =  50
Input sentence: Spouse Off—Job Acc June 18, 2012 
GT sentence: Spouse Off-Job Acc June 18, 2012
Char Decoded sentence: Spouse Off—Job Acc June 18, 2012
Word Decoded sentence: Spouse Off—Job Acc June 18, 2012 
-Lenght =  50
Input sentence: Child Off—Job Acc June 18, 2012 
GT sentence: Child Off-Job Acc June 18, 2012
Char Decoded sentence: Child Off—Job Acc June 18, 2012
Word Decoded sentence: Child Off—Job Acc June 18, 2012 
-Lenght =  50
Input sentence: Total Monthly Premium: $31.81 
GT sentence: Total Monthly Premium: $31.81
Char Decoded sentence: Total Monthly Premium: $31.81
Word Decoded sentence: Total Monthly Premium $31.81 
-Lenght =  100
Input sentence: Total Employee Semi-Monthly Payroll Deduction: $15.92 
GT sentence: Total Employee Sem

In [19]:
WER_spell_correction = calculate_WER(gt_texts, decoded_sentences)
print('WER_spell_correction |TEST= ', WER_spell_correction)

WER_spell_correction |TEST=  0.113000765838


In [20]:
WER_spell_word_correction = calculate_WER(gt_texts, corrected_sentences)
print('WER_spell_word_correction |TEST= ', WER_spell_word_correction)

WER_spell_word_correction |TEST=  0.129743576201


In [21]:
WER_OCR = calculate_WER(gt_texts, input_texts)
print('WER_OCR |TEST= ', WER_OCR)

WER_OCR |TEST=  0.0563287295006
